## Doublet Removal - Pseudo Doublets

In [3]:
import scanpy as sc
import numpy as np
import pandas as pd
from matplotlib import rcParams
import matplotlib.pyplot as plt
%matplotlib inline
import scipy
import os
import anndata

sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

# data files are in the filtered matrix folder from CellRanger which has been renamed to the sample name
sample_name = r'sc85_3g' #r'sc70_1' #r'sc72_1'
data_path = r"../raw_data/"
data_files_path = data_path + sample_name + r""

results_path = 'results/'
results_file = results_path + sample_name + '-db.h5ad'  # the file that will store the analysis results
metrics_file = results_path + sample_name + '_db_metrics.csv'  # the file that will store the metrics
# make results folder if it doesn't exist
if not os.path.exists(results_path):
    os.makedirs(results_path, exist_ok=False)

scanpy==1.10.3 anndata==0.11.0 umap==0.5.7 numpy==1.26.4 scipy==1.14.1 pandas==2.2.3 scikit-learn==1.5.2 statsmodels==0.14.4 igraph==0.11.8 louvain==0.8.2 pynndescent==0.5.13


#### Setup R environment

In [4]:
import os
os.environ['R_HOME'] = r"C:\Program Files\R\R-4.4.2"  
import anndata2ri # order matters, comes after defining 'R_HOME
import logging
import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro
rcb.logger.setLevel(logging.ERROR)
ro.pandas2ri.activate()
anndata2ri.activate()

%reload_ext rpy2.ipython

C:\Users\leeh1\AppData\Local\Temp\ipykernel_13112\364680306.py:9: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  anndata2ri.activate()
C:\Users\leeh1\McKinnonRosatiLab\jl\lib\site-packages\rpy2\robjects\packages.py:367: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


In [5]:
%%R
.libPaths(c("C:/Users/leeh1/AppData/Local/R/win-library/4.4", .libPaths()))
library(Seurat)
library(scater)
library(scDblFinder)
library(BiocParallel)


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

Loading required package: SeuratObject
Loading required package: sp

Attaching package: 'SeuratObject'

The following objects are masked from 'package:base':

    intersect, t

Loading required package: SingleCellExperiment
Loading required package: SummarizedExperiment
Loading required package: MatrixGenerics
Loading required package: matrixStats

Attaching package: 'MatrixGenerics'

The following objects are masked from 'package:matrixStats':

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts,

#### Load data

Read in count data into an [`AnnData`](https://anndata.readthedocs.io/en/latest/anndata.AnnData.html) object.  
The matrix.mtx file contains a sparse matrix of the counts.  
Barcodes file contains the sample names in the format: AAACCCAAGACCATAA-1  
Features file contains the gene id in the format: ENSMUSG00000051951, Xkr4, Gene Expression

The number of counts per cell will be relatively low since only UMIs are counted by Cellranger

In [6]:
adata = sc.read_10x_mtx(
    data_files_path,              # the directory with the `.mtx` file
    var_names='gene_symbols',     # use gene symbols for the variable names (variables-axis index)
    cache=True)                   # write a cache file for faster subsequent reading

# adata.var_names_make_unique()  # this is unnecessary if using `var_names='gene_ids'` in `sc.read_10x_mtx`
print('Data matrix is sparse:', scipy.sparse.issparse(adata.X))
print()

# make the obs names unique by adding the sample name
adata.obs_names = [g.split("-")[0] + '_' + sample_name for g in adata.obs_names]
print(adata.obs_names[0:2])
print()
print('Number of cells =', f"{adata.n_obs:,.0f}")
print('Number of genes =', f"{adata.n_vars:,.0f}")
print('Number of counts =', f"{adata.X.sum():,.0f}")
print('Mean counts per cell =', f"{adata.X.sum()/adata.n_obs:,.0f}")
adata

... reading from cache file cache\..-raw_data-sc85_3g-matrix.h5ad
Data matrix is sparse: True

Index(['AAACAGCCAACATAAG_sc85_3g', 'AAACAGCCAATGAAGC_sc85_3g'], dtype='object')

Number of cells = 13,572
Number of genes = 36,530
Number of counts = 18,226,496
Mean counts per cell = 1,343


AnnData object with n_obs × n_vars = 13572 × 36530
    var: 'gene_ids', 'feature_types'

In [7]:
n_cells = adata.n_obs
n_genes = adata.n_vars
n_counts = adata.X.sum()    
counts_per_cell = round(n_counts / n_cells)

df_metrics = pd.DataFrame([n_cells, n_genes, n_counts, counts_per_cell], \
                  index = ['Number of cells', 'Number of genes', 'Number of counts','Mean counts per cell']).T
df_metrics.iloc[:, 1:] = df_metrics.iloc[:,1:].applymap('{:,.0f}'.format)
df_metrics.index = [sample_name]
df_metrics

C:\Users\leeh1\AppData\Local\Temp\ipykernel_13112\1815972615.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_metrics.iloc[:, 1:] = df_metrics.iloc[:,1:].applymap('{:,.0f}'.format)
C:\Users\leeh1\AppData\Local\Temp\ipykernel_13112\1815972615.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    36,530
Name: Number of genes, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_metrics.iloc[:, 1:] = df_metrics.iloc[:,1:].applymap('{:,.0f}'.format)
C:\Users\leeh1\AppData\Local\Temp\ipykernel_13112\1815972615.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    18,226,496
Name: Number of counts, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_metrics.iloc[:, 1:] = df_metrics.iloc[:,1

,Number of cells,Number of genes,Number of counts,Mean counts per cell
sc85_3g,13572.0,"36,530","18,226,496","1,343"


### Import Doublet IDs

In [8]:
doublets = np.load('doublets_sc85_3g.npy')
print(len(doublets))

4197


### Isolating Singlets

In [9]:
adata.obs['doublet_class'] = [1.0 if name in doublets else 0.0 for name in adata.obs_names]

In [10]:
print(adata.obs['doublet_class'])

AAACAGCCAACATAAG_sc85_3g    1.0
AAACAGCCAATGAAGC_sc85_3g    0.0
AAACAGCCAATTAAGG_sc85_3g    1.0
AAACAGCCACGTGCTG_sc85_3g    0.0
AAACAGCCACTGACCG_sc85_3g    1.0
                           ... 
TTTGTGTTCTAGCTTT_sc85_3g    1.0
TTTGTGTTCTGTTCAT_sc85_3g    0.0
TTTGTTGGTCCTTAGT_sc85_3g    1.0
TTTGTTGGTGTCCAGG_sc85_3g    0.0
TTTGTTGGTTGTCATC_sc85_3g    0.0
Name: doublet_class, Length: 13572, dtype: float64


In [11]:
bdata = adata[adata.obs['doublet_class'] == 0.0].copy()

In [12]:
print(len(bdata))

9375


## Creating Pseudo Doublets

In [13]:
singlet_data = bdata.X.T
n_pseudo_doubs = int(0.1 * bdata.shape[0])
print("Number of pseudo_doublets to be generated: " + str(n_pseudo_doubs))

Number of pseudo_doublets to be generated: 937


In [14]:
%%R -i singlet_data -i n_pseudo_doubs -o pseudo_doublets -o pd_count_matrix -o names

pseudo_doublets = getArtificialDoublets(
    x=singlet_data,   
    n=n_pseudo_doubs,  
    resamp=0.1, 
    propRandom=0,  
    selMode="proportional",  
    meta.triplets=FALSE  
)

pd_count_matrix <- pseudo_doublets$counts
str(pd_count_matrix)
names <- pd_count_matrix@Dimnames[[2]]
str(names)

Formal class 'dgCMatrix' [package "Matrix"] with 6 slots
  ..@ i       : int [1:1324989] 44 50 65 74 79 93 104 116 127 154 ...
  ..@ p       : int [1:938] 0 1608 3226 4677 5464 6913 8638 9893 11778 13046 ...
  ..@ Dim     : int [1:2] 36530 937
  ..@ Dimnames:List of 2
  .. ..$ : NULL
  .. ..$ : chr [1:937] "rDbl.1" "rDbl.2" "rDbl.3" "rDbl.4" ...
  ..@ x       : num [1:1324989] 1 2 1 1 2 1 1 1 1 1 ...
  ..@ factors : list()
 chr [1:937] "rDbl.1" "rDbl.2" "rDbl.3" "rDbl.4" "rDbl.5" "rDbl.6" "rDbl.7" ...


In [15]:
print("pd_count_matrix shape:", pd_count_matrix)

pd_count_matrix shape: <Compressed Sparse Column sparse matrix of dtype 'float64'
	with 1324989 stored elements and shape (36530, 937)>
  Coords	Values
  (44, 0)	1.0
  (50, 0)	2.0
  (65, 0)	1.0
  (74, 0)	1.0
  (79, 0)	2.0
  (93, 0)	1.0
  (104, 0)	1.0
  (116, 0)	1.0
  (127, 0)	1.0
  (154, 0)	1.0
  (179, 0)	3.0
  (180, 0)	1.0
  (195, 0)	1.0
  (229, 0)	1.0
  (233, 0)	2.0
  (238, 0)	1.0
  (269, 0)	1.0
  (272, 0)	1.0
  (273, 0)	1.0
  (299, 0)	2.0
  (300, 0)	1.0
  (322, 0)	1.0
  (337, 0)	1.0
  (373, 0)	1.0
  (403, 0)	1.0
  :	:
  (34664, 936)	0.5
  (34665, 936)	1.5
  (34666, 936)	1.0
  (34671, 936)	0.5
  (34674, 936)	0.5
  (34675, 936)	0.5
  (34707, 936)	0.5
  (34779, 936)	1.5
  (34791, 936)	0.5
  (34892, 936)	0.5
  (35553, 936)	0.5
  (35555, 936)	0.5
  (35697, 936)	0.5
  (36262, 936)	0.5
  (36310, 936)	0.5
  (36493, 936)	36.0
  (36495, 936)	27.5
  (36497, 936)	2.0
  (36501, 936)	3.0
  (36507, 936)	1.0
  (36510, 936)	8.5
  (36513, 936)	3.5
  (36514, 936)	3.0
  (36519, 936)	2.0
  (36526, 936)	

In [16]:
print("names", names)

names ['rDbl.1' 'rDbl.2' 'rDbl.3' 'rDbl.4' 'rDbl.5' 'rDbl.6' 'rDbl.7' 'rDbl.8'
 'rDbl.9' 'rDbl.10' 'rDbl.11' 'rDbl.12' 'rDbl.13' 'rDbl.14' 'rDbl.15'
 'rDbl.16' 'rDbl.17' 'rDbl.18' 'rDbl.19' 'rDbl.20' 'rDbl.21' 'rDbl.22'
 'rDbl.23' 'rDbl.24' 'rDbl.25' 'rDbl.26' 'rDbl.27' 'rDbl.28' 'rDbl.29'
 'rDbl.30' 'rDbl.31' 'rDbl.32' 'rDbl.33' 'rDbl.34' 'rDbl.35' 'rDbl.36'
 'rDbl.37' 'rDbl.38' 'rDbl.39' 'rDbl.40' 'rDbl.41' 'rDbl.42' 'rDbl.43'
 'rDbl.44' 'rDbl.45' 'rDbl.46' 'rDbl.47' 'rDbl.48' 'rDbl.49' 'rDbl.50'
 'rDbl.51' 'rDbl.52' 'rDbl.53' 'rDbl.54' 'rDbl.55' 'rDbl.56' 'rDbl.57'
 'rDbl.58' 'rDbl.59' 'rDbl.60' 'rDbl.61' 'rDbl.62' 'rDbl.63' 'rDbl.64'
 'rDbl.65' 'rDbl.66' 'rDbl.67' 'rDbl.68' 'rDbl.69' 'rDbl.70' 'rDbl.71'
 'rDbl.72' 'rDbl.73' 'rDbl.74' 'rDbl.75' 'rDbl.76' 'rDbl.77' 'rDbl.78'
 'rDbl.79' 'rDbl.80' 'rDbl.81' 'rDbl.82' 'rDbl.83' 'rDbl.84' 'rDbl.85'
 'rDbl.86' 'rDbl.87' 'rDbl.88' 'rDbl.89' 'rDbl.90' 'rDbl.91' 'rDbl.92'
 'rDbl.93' 'rDbl.94' 'rDbl.95' 'rDbl.96' 'rDbl.97' 'rDbl.98' 'rDbl.99'

#### Checking `bdata`

In [17]:
print(bdata.X)

<Compressed Sparse Column sparse matrix of dtype 'float32'
	with 7534568 stored elements and shape (9375, 36530)>
  Coords	Values
  (61, 0)	1.0
  (4455, 0)	1.0
  (4613, 0)	1.0
  (5255, 0)	1.0
  (587, 3)	1.0
  (3198, 3)	1.0
  (3361, 3)	1.0
  (4274, 3)	1.0
  (5514, 3)	1.0
  (4259, 8)	1.0
  (5311, 8)	1.0
  (17, 11)	1.0
  (29, 11)	1.0
  (1175, 11)	1.0
  (1361, 11)	1.0
  (1461, 11)	1.0
  (1905, 11)	1.0
  (1937, 11)	1.0
  (2031, 11)	1.0
  (2362, 11)	1.0
  (2376, 11)	1.0
  (2396, 11)	1.0
  (2521, 11)	1.0
  (2783, 11)	1.0
  (2885, 11)	1.0
  :	:
  (4975, 36529)	1.0
  (5020, 36529)	1.0
  (5097, 36529)	1.0
  (5214, 36529)	1.0
  (5250, 36529)	1.0
  (5348, 36529)	1.0
  (5746, 36529)	1.0
  (6132, 36529)	1.0
  (6244, 36529)	1.0
  (6347, 36529)	1.0
  (6804, 36529)	1.0
  (6866, 36529)	1.0
  (7108, 36529)	2.0
  (7114, 36529)	1.0
  (7298, 36529)	1.0
  (7447, 36529)	1.0
  (7578, 36529)	1.0
  (7662, 36529)	1.0
  (7718, 36529)	1.0
  (7870, 36529)	1.0
  (8148, 36529)	1.0
  (8195, 36529)	1.0
  (8221, 36529)	1

#### Creating `pseudo_doublet_adata`

In [18]:
import anndata
pseudo_doublet_adata = anndata.AnnData(pd_count_matrix)
pseudo_doublet_adata = pseudo_doublet_adata.T
pseudo_doublet_adata.obs_names = names
pseudo_doublet_adata.var_names = bdata.var_names
pseudo_doublet_adata.obs["doublet"] = 1.0

In [19]:
print(pseudo_doublet_adata.shape)
print(pseudo_doublet_adata.obs_names)
print(pseudo_doublet_adata.var_names)
print(pseudo_doublet_adata.obs)
print(pseudo_doublet_adata.X)

(937, 36530)
Index(['rDbl.1', 'rDbl.2', 'rDbl.3', 'rDbl.4', 'rDbl.5', 'rDbl.6', 'rDbl.7',
       'rDbl.8', 'rDbl.9', 'rDbl.10',
       ...
       'rDbl.928', 'rDbl.929', 'rDbl.930', 'rDbl.931', 'rDbl.932', 'rDbl.933',
       'rDbl.934', 'rDbl.935', 'rDbl.936', 'rDbl.937'],
      dtype='object', length=937)
Index(['LOC139071432', 'LOC139071438', 'LOC139072265', 'LOC107385520',
       'LOC129157067', 'LOC139072266', 'LOC139070601', 'LOC129164336',
       'LOC139072245', 'LOC139063175',
       ...
       'KEG92_t17', 'KEG92_t18', 'KEG92_t19', 'KEG92_p03', 'KEG92_p02',
       'KEG92_t20', 'KEG92_p01', 'KEG92_t21', 'KEG92_t22', 'KEG92_t23'],
      dtype='object', length=36530)
          doublet
rDbl.1        1.0
rDbl.2        1.0
rDbl.3        1.0
rDbl.4        1.0
rDbl.5        1.0
...           ...
rDbl.933      1.0
rDbl.934      1.0
rDbl.935      1.0
rDbl.936      1.0
rDbl.937      1.0

[937 rows x 1 columns]
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1324989 stored el

#### Concatenating adata objects

In [20]:
adata_combined = anndata.concat([pseudo_doublet_adata, bdata], axis=0, join="inner")

In [21]:
print(adata_combined.shape)

(10312, 36530)


In [22]:
print(adata_combined.obs)
print(adata_combined.var)
print(adata_combined.X)

Empty DataFrame
Columns: []
Index: [rDbl.1, rDbl.2, rDbl.3, rDbl.4, rDbl.5, rDbl.6, rDbl.7, rDbl.8, rDbl.9, rDbl.10, rDbl.11, rDbl.12, rDbl.13, rDbl.14, rDbl.15, rDbl.16, rDbl.17, rDbl.18, rDbl.19, rDbl.20, rDbl.21, rDbl.22, rDbl.23, rDbl.24, rDbl.25, rDbl.26, rDbl.27, rDbl.28, rDbl.29, rDbl.30, rDbl.31, rDbl.32, rDbl.33, rDbl.34, rDbl.35, rDbl.36, rDbl.37, rDbl.38, rDbl.39, rDbl.40, rDbl.41, rDbl.42, rDbl.43, rDbl.44, rDbl.45, rDbl.46, rDbl.47, rDbl.48, rDbl.49, rDbl.50, rDbl.51, rDbl.52, rDbl.53, rDbl.54, rDbl.55, rDbl.56, rDbl.57, rDbl.58, rDbl.59, rDbl.60, rDbl.61, rDbl.62, rDbl.63, rDbl.64, rDbl.65, rDbl.66, rDbl.67, rDbl.68, rDbl.69, rDbl.70, rDbl.71, rDbl.72, rDbl.73, rDbl.74, rDbl.75, rDbl.76, rDbl.77, rDbl.78, rDbl.79, rDbl.80, rDbl.81, rDbl.82, rDbl.83, rDbl.84, rDbl.85, rDbl.86, rDbl.87, rDbl.88, rDbl.89, rDbl.90, rDbl.91, rDbl.92, rDbl.93, rDbl.94, rDbl.95, rDbl.96, rDbl.97, rDbl.98, rDbl.99, rDbl.100, ...]

[10312 rows x 0 columns]
Empty DataFrame
Columns: []
Index: [LOC13

In [23]:
adata_combined.write("pseudo_doublets_sc85_3g_2.0.h5ad")

### Exporting Pseudo Doublets

In [24]:
with open("./sc85_3g_IDs/pseudo_sc85_3g_2.0_ids.txt", "w") as txt_file:
    for cell in names:
        txt_file.write(cell + "\n")